In [43]:
import json 
with open('intents.json') as file:
    data = json.load(file)

In [47]:
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)
print(num_classes,labels)
print(training_sentences)
print(responses)

8 ['greeting', 'goodbye', 'thanks', 'name', 'about', 'interest', 'seeking', 'skills']
['Hi', 'Hey', 'Is anyone there?', 'Hello', 'Hay', 'Bye', 'See you later', 'Goodbye', 'good night', 'Thanks', 'Thank you', "That's helpful", 'Thanks for the help', 'Who are you?', 'What are you?', 'Who you are?', 'what is your name', 'what should I call you', 'whats your name?', 'tell me more about yourself', "I'd like to know more about you", 'give me more information on you', 'share further details about yourself', 'Tell me a little bit more about yourself', 'What are your areas of interest?', 'your passion', 'what are your hobbies', 'your hobbies', 'what are subjects of your interest', 'What sparks your interest?', 'what do want to study more', 'what do you want to learn', 'what are you seeking', 'What are your current goals or objectives?', 'what do you want to do now', 'what are you aiming for', 'what are your aspirations', 'what do you want to do know', 'your skills', 'what are you skilled in', '

In [48]:
from sklearn.preprocessing import LabelEncoder

lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)

# label encoder change the intents into numerical categories i.e 0,1,2,3,4,5,6,7
training_labels = lbl_encoder.transform(training_labels)
training_labels

array([2, 2, 2, 2, 2, 1, 1, 1, 1, 7, 7, 7, 7, 4, 4, 4, 4, 4, 4, 0, 0, 0,
       0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6,
       6])

In [52]:

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 1000
embedding_dim = 16
max_len = 20
# adding this deals with out of vocabulary words
oov_token = "<OOV>"

# adding out of vocabulary token
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token) 
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)

# makes all training text sequences into same size
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)
word_index

{'<OOV>': 1,
 'what': 2,
 'you': 3,
 'are': 4,
 'your': 5,
 'do': 6,
 'more': 7,
 'about': 8,
 'to': 9,
 'me': 10,
 'want': 11,
 'tell': 12,
 'yourself': 13,
 'interest': 14,
 'is': 15,
 'good': 16,
 'thanks': 17,
 'for': 18,
 'who': 19,
 'name': 20,
 'know': 21,
 'areas': 22,
 'of': 23,
 'hobbies': 24,
 'skills': 25,
 'hi': 26,
 'hey': 27,
 'anyone': 28,
 'there': 29,
 'hello': 30,
 'hay': 31,
 'bye': 32,
 'see': 33,
 'later': 34,
 'goodbye': 35,
 'night': 36,
 'thank': 37,
 "that's": 38,
 'helpful': 39,
 'the': 40,
 'help': 41,
 'should': 42,
 'i': 43,
 'call': 44,
 'whats': 45,
 "i'd": 46,
 'like': 47,
 'give': 48,
 'information': 49,
 'on': 50,
 'share': 51,
 'further': 52,
 'details': 53,
 'a': 54,
 'little': 55,
 'bit': 56,
 'passion': 57,
 'subjects': 58,
 'sparks': 59,
 'study': 60,
 'learn': 61,
 'seeking': 62,
 'current': 63,
 'goals': 64,
 'or': 65,
 'objectives': 66,
 'now': 67,
 'aiming': 68,
 'aspirations': 69,
 'skilled': 70,
 'in': 71,
 'at': 72,
 'strong': 73,
 'can': 

In [53]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# difference between sparse crossentropy and crossentropy is that in former labels are expected to be provided in integers values i.e 0,1,2,3
model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [54]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d_5   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_15 (Dense)            (None, 16)                272       
                                                                 
 dense_16 (Dense)            (None, 16)                272       
                                                                 
 dense_17 (Dense)            (None, 8)                 136       
                                                                 
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
__________________________________________________

In [55]:
epochs = 600
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/1000
2/2 [==============================] - 2s 16ms/step - loss: 2.0795 - accuracy: 0.1333
Epoch 2/1000
2/2 [==============================] - 0s 8ms/step - loss: 2.0789 - accuracy: 0.0889
Epoch 3/1000
2/2 [==============================] - 0s 10ms/step - loss: 2.0781 - accuracy: 0.1111
Epoch 4/1000
2/2 [==============================] - 0s 10ms/step - loss: 2.0777 - accuracy: 0.2222
Epoch 5/1000
2/2 [==============================] - 0s 16ms/step - loss: 2.0771 - accuracy: 0.2222
Epoch 6/1000
2/2 [==============================] - 0s 16ms/step - loss: 2.0764 - accuracy: 0.1556
Epoch 7/1000
2/2 [==============================] - 0s 8ms/step - loss: 2.0759 - accuracy: 0.1556
Epoch 8/1000
2/2 [==============================] - 0s 8ms/step - loss: 2.0754 - accuracy: 0.1556
Epoch 9/1000
2/2 [==============================] - 0s 8ms/step - loss: 2.0747 - accuracy: 0.1556
Epoch 10/1000
2/2 [==============================] - 0s 15ms/step - loss: 2.0741 - accuracy: 0.1556
Epoch 11/1000

In [56]:
# saving model
model.save("chat_model")

import pickle

# saving tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

# saving label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model\assets
